In [1]:
import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks
from scipy import signal
from scipy.optimize import curve_fit
import cmath
import control
from control.matlab import *
plt.style.use('seaborn-darkgrid')
import h5py
from lmfit.models import LorentzianModel, QuadraticModel,LinearModel
import matplotlib.pyplot as plt
from numpy import exp, loadtxt, pi, sqrt
from lmfit import Model

In [11]:
import hdf5storage
folder = 'data'
file_name = 'final_data.mat'
data = hdf5storage.loadmat(f'{folder}/{file_name}')
data.keys()

dict_keys(['XX', 'YY', 'f0', 'name2Save', 'sampleintervalS'])

In [9]:
new_centers = [5412.808271713509, 5612.11038571605, 6324.503304154022, 6380.4417622063465, 6406.121505718675, 6861.694279189701, 7077.525028825343, 7586.608794535136, 8251.356047802785, 8557.913141831705, 8881.526339770082, 8981.335585820168, 9224.594123834742, 10274.944365581925, 10293.965950948092, 10346.69327582523, 12732.42370972809, 13000.0]

### RUNNING ALL THE POINTS

In [12]:
Fc_low = 13500
Fc_high = 1000
L = 1880
Fs = 199999

import json
import datetime

results = {}

for i in range(2): #goes through columns
    if  i % 4 == 0 :
        cut_file = i
        results2 = results.copy()
        results = {}
    for j in range(118): #goes through rows
        
        if f'[{i},{j}]' not in results2.keys():
            print(f'[{i},{j}]')
            results[f'[{i},{j}]'] = {'params': [] , 'red_chi': [],'chi_sq': []}
            v = data['ZZ'][:,i,j][120:2000]

            sos_l = signal.bessel(5,Fc_low, 'low', fs=Fs,output='sos') # LPF filtering parameters
            sos_h = signal.bessel(5,Fc_high, 'high', fs=Fs,output='sos') # HPF filtering parameters

            x = signal.sosfilt(sos_l,v) # LPF
            x = signal.sosfilt(sos_h,x) # HPF

            freqs=np.fft.fftfreq(L,1/Fs)
            f = freqs[freqs>100] #frequency vector (x-axis in frec domain)
            fft= np.fft.fft(x)[freqs>100]
            
            if np.abs(max(fft)) >30000: #threshold for the amplitude (now passes abput 18% of all the points)
                print('Nuevo:',i,j,np.abs(max(fft)))

                mod = Model(MyLorentzian2) #here I call the name function of the model I want to create
                pars = mod.make_params()
                for n,p in enumerate(new_centers):
                    index_p = np.where(f>p)[0][0]
                    prefix = 'lz%d_' % (n+1)
                    pars.add(name = f'{prefix}center', value = f[index_p],vary=False)
                    mod.set_param_hint(f'{prefix}center', vary=False) #this tell the model to fix the parameter
                    pars.add(name = f'{prefix}sigma',value =  5, min=0)
                    pars.add(name = f'{prefix}amplitude_real', value = fft[index_p].real)
                    pars.add(name = f'{prefix}amplitude_imag', value = fft[index_p].imag)

                t1 = datetime.datetime.now() #just estimating the time it takes to the program to fit
                result = mod.fit(np.abs(fft), pars, x=f,nan_policy='omit') #THIS IS THE FIT
                t2 = datetime.datetime.now()
                params_ = list(result.values.values())
                results[f'[{i},{j}]']['params'] = list(params_)
                results[f'[{i},{j}]']['red_chi']= result.redchi
                results[f'[{i},{j}]']['chi_sq'] = result.chisqr

                
                print(f'Time: {t2-t1}')
                with open(f'new_results2/{cut_file + 1}.txt', 'w') as file:
                    file.truncate(0)
                    file.write(json.dumps(results)) # use `json.loads` to do the reverse

        #print(result.fit_report())

[0,0]


KeyError: 'ZZ'

In [29]:
mod

<lmfit.Model: Model(MyLorentzian2)>

### Functions

In [9]:

def Lorentzian(x,lz1_center,lz1_sigma,lz1_amplitude_real,lz1_amplitude_imag):
    """returns the absolute values of the sum of 13 lorentzian function"""
    
    lz1_amplitude =  lz1_amplitude_real + 1j*lz1_amplitude_imag

    l1 = (lz1_amplitude * lz1_sigma**2) / ( lz1_sigma**2 + ( x - lz1_center )**2) 

    
    return l1

In [3]:
def MyLorentzian2(x,lz1_center,lz1_sigma,lz1_amplitude_real,lz1_amplitude_imag,lz2_center,lz2_sigma,lz2_amplitude_real,lz2_amplitude_imag,lz3_center,lz3_sigma,lz3_amplitude_real,lz3_amplitude_imag,lz4_center,lz4_sigma,lz4_amplitude_real,lz4_amplitude_imag,lz5_center,lz5_sigma,lz5_amplitude_real,lz5_amplitude_imag,lz6_center,lz6_sigma,lz6_amplitude_real,lz6_amplitude_imag,lz7_center,lz7_sigma,lz7_amplitude_real,lz7_amplitude_imag,lz8_center,lz8_sigma,lz8_amplitude_real,lz8_amplitude_imag,lz9_center,lz9_sigma,lz9_amplitude_real,lz9_amplitude_imag,lz10_center,lz10_sigma,lz10_amplitude_real,lz10_amplitude_imag,lz11_center,lz11_sigma,lz11_amplitude_real,lz11_amplitude_imag,lz12_center,lz12_sigma,lz12_amplitude_real,lz12_amplitude_imag,lz13_center,lz13_sigma,lz13_amplitude_real,lz13_amplitude_imag,lz14_center,lz14_sigma,lz14_amplitude_real,lz14_amplitude_imag,lz15_center,lz15_sigma,lz15_amplitude_real,lz15_amplitude_imag,lz16_center,lz16_sigma,lz16_amplitude_real,lz16_amplitude_imag,lz17_center,lz17_sigma,lz17_amplitude_real,lz17_amplitude_imag,lz18_center,lz18_sigma,lz18_amplitude_real,lz18_amplitude_imag):
    """returns the absolute values of the sum of 13 lorentzian function"""
    
    lz1_amplitude =  lz1_amplitude_real + 1j*lz1_amplitude_imag
    lz2_amplitude =  lz2_amplitude_real + 1j*lz2_amplitude_imag
    lz3_amplitude =  lz3_amplitude_real + 1j*lz3_amplitude_imag
    lz4_amplitude =  lz4_amplitude_real + 1j*lz4_amplitude_imag
    lz5_amplitude =  lz5_amplitude_real + 1j*lz5_amplitude_imag
    lz6_amplitude =  lz6_amplitude_real + 1j*lz6_amplitude_imag
    lz7_amplitude =  lz7_amplitude_real + 1j*lz7_amplitude_imag
    lz8_amplitude =  lz8_amplitude_real + 1j*lz8_amplitude_imag
    lz9_amplitude =  lz9_amplitude_real + 1j*lz9_amplitude_imag
    lz10_amplitude =  lz10_amplitude_real + 1j*lz10_amplitude_imag
    lz11_amplitude =  lz11_amplitude_real + 1j*lz11_amplitude_imag
    lz12_amplitude =  lz12_amplitude_real + 1j*lz12_amplitude_imag
    lz13_amplitude =  lz13_amplitude_real + 1j*lz13_amplitude_imag
    lz14_amplitude =  lz14_amplitude_real + 1j*lz14_amplitude_imag
    lz15_amplitude =  lz15_amplitude_real + 1j*lz15_amplitude_imag
    lz16_amplitude =  lz16_amplitude_real + 1j*lz16_amplitude_imag
    lz17_amplitude =  lz17_amplitude_real + 1j*lz17_amplitude_imag
    lz18_amplitude =  lz18_amplitude_real + 1j*lz18_amplitude_imag


    l1 = (lz1_amplitude * lz1_sigma**2) / ( lz1_sigma**2 + ( x - lz1_center )**2) 
    l2 = (lz2_amplitude * lz2_sigma**2) / ( lz2_sigma**2 + ( x - lz2_center )**2) 
    l3 = (lz3_amplitude * lz3_sigma**2) / ( lz3_sigma**2 + ( x - lz3_center )**2)
    l4 = (lz4_amplitude * lz4_sigma**2) / ( lz4_sigma**2 + ( x - lz4_center )**2) 
    l5 = (lz5_amplitude * lz5_sigma**2) / ( lz5_sigma**2 + ( x - lz5_center )**2)
    l6 = (lz6_amplitude * lz6_sigma**2) / ( lz6_sigma**2 + ( x - lz6_center )**2) 
    l7 = (lz7_amplitude * lz7_sigma**2) / ( lz7_sigma**2 + ( x - lz7_center )**2) 
    l8 = (lz8_amplitude * lz8_sigma**2) / ( lz8_sigma**2 + ( x - lz8_center )**2)
    l9 = (lz9_amplitude * lz9_sigma**2) / ( lz9_sigma**2 + ( x - lz9_center )**2) 
    l10 = (lz10_amplitude * lz10_sigma**2) / ( lz10_sigma**2 + ( x - lz10_center )**2) 
    l11 = (lz11_amplitude * lz11_sigma**2) / ( lz11_sigma**2 + ( x - lz11_center )**2) 
    l12 = (lz12_amplitude * lz12_sigma**2) / ( lz12_sigma**2 + ( x - lz12_center )**2) 
    l13 = (lz13_amplitude * lz13_sigma**2) / ( lz13_sigma**2 + ( x - lz13_center )**2)
    l14 = (lz14_amplitude * lz14_sigma**2) / ( lz14_sigma**2 + ( x - lz14_center )**2) 
    l15 = (lz15_amplitude * lz15_sigma**2) / ( lz15_sigma**2 + ( x - lz15_center )**2)
    l16 = (lz16_amplitude * lz16_sigma**2) / ( lz16_sigma**2 + ( x - lz16_center )**2) 
    l17 = (lz17_amplitude * lz17_sigma**2) / ( lz17_sigma**2 + ( x - lz17_center )**2) 
    l18 = (lz18_amplitude * lz18_sigma**2) / ( lz18_sigma**2 + ( x - lz18_center )**2)
    
    return np.abs(l1+l2+l3+l4+l5+l6+l7+l8+l9+l10+l11+l12+l13+l14+l15+l16+l17+l18)


def MyLorentzian10(x,lz1_center,lz1_sigma,lz1_amplitude_real,lz1_amplitude_imag,lz2_center,lz2_sigma,lz2_amplitude_real,lz2_amplitude_imag,lz3_center,lz3_sigma,lz3_amplitude_real,lz3_amplitude_imag,lz4_center,lz4_sigma,lz4_amplitude_real,lz4_amplitude_imag,lz5_center,lz5_sigma,lz5_amplitude_real,lz5_amplitude_imag,lz6_center,lz6_sigma,lz6_amplitude_real,lz6_amplitude_imag,lz7_center,lz7_sigma,lz7_amplitude_real,lz7_amplitude_imag,lz8_center,lz8_sigma,lz8_amplitude_real,lz8_amplitude_imag,lz9_center,lz9_sigma,lz9_amplitude_real,lz9_amplitude_imag,lz10_center,lz10_sigma,lz10_amplitude_real,lz10_amplitude_imag):
    """returns the absolute values of the sum of 13 lorentzian function"""
    
    lz1_amplitude =  lz1_amplitude_real + 1j*lz1_amplitude_imag
    lz2_amplitude =  lz2_amplitude_real + 1j*lz2_amplitude_imag
    lz3_amplitude =  lz3_amplitude_real + 1j*lz3_amplitude_imag
    lz4_amplitude =  lz4_amplitude_real + 1j*lz4_amplitude_imag
    lz5_amplitude =  lz5_amplitude_real + 1j*lz5_amplitude_imag
    lz6_amplitude =  lz6_amplitude_real + 1j*lz6_amplitude_imag
    lz7_amplitude =  lz7_amplitude_real + 1j*lz7_amplitude_imag
    lz8_amplitude =  lz8_amplitude_real + 1j*lz8_amplitude_imag
    lz9_amplitude =  lz9_amplitude_real + 1j*lz9_amplitude_imag
    lz10_amplitude =  lz10_amplitude_real + 1j*lz10_amplitude_imag


    l1 = (lz1_amplitude * lz1_sigma**2) / ( lz1_sigma**2 + ( x - lz1_center )**2) 
    l2 = (lz2_amplitude * lz2_sigma**2) / ( lz2_sigma**2 + ( x - lz2_center )**2) 
    l3 = (lz3_amplitude * lz3_sigma**2) / ( lz3_sigma**2 + ( x - lz3_center )**2)
    l4 = (lz4_amplitude * lz4_sigma**2) / ( lz4_sigma**2 + ( x - lz4_center )**2) 
    l5 = (lz5_amplitude * lz5_sigma**2) / ( lz5_sigma**2 + ( x - lz5_center )**2)
    l6 = (lz6_amplitude * lz6_sigma**2) / ( lz6_sigma**2 + ( x - lz6_center )**2) 
    l7 = (lz7_amplitude * lz7_sigma**2) / ( lz7_sigma**2 + ( x - lz7_center )**2) 
    l8 = (lz8_amplitude * lz8_sigma**2) / ( lz8_sigma**2 + ( x - lz8_center )**2)
    l9 = (lz9_amplitude * lz9_sigma**2) / ( lz9_sigma**2 + ( x - lz9_center )**2) 
    l10 = (lz10_amplitude * lz10_sigma**2) / ( lz10_sigma**2 + ( x - lz10_center )**2) 


    
    return np.abs(l1+l2+l3+l4+l5+l6+l7+l8+l9+l10)

In [91]:
def norm(x,y):
    """euclidean distance"""
    return np.sqrt(x**2 + y**2)

def consecutive_peaks(peak_index):
    """returns in sublists the consecutive indexes"""
    sub_list = np.split(peak_index, np.where(np.diff(peak_index) > 1)[0] + 1)
    return sub_list
   
    
def find_near_peaks(x,y,peaks):
    """ gets the indexes of the peaks that are 'near' each other. 
    'near': the distance beaing a higher order than the average . 
    Can be modified depending on the noise of the data (as it creates micro-peaks)
    """
    near_peaks = []
    peak_dist = norm(np.diff(x[peaks]),np.diff(y[peaks]))  #distance between each consecutive peak
    avg_peak_dist = np.mean(peak_dist)*10 # 1 higher order than mean of distance between peaks 
    for i,peak in enumerate(peaks):
        if i == 0:
            dist_next = norm(np.diff([x[peak],x[peaks[i+1]]]),np.diff([y[peak],y[peaks[i+1]]]))
            if dist_next < avg_peak_dist:
                near_peaks.append(peak)
        elif i == len(peaks)-1:
            dist_prev = norm(np.diff([x[peak],x[peaks[i-1]]]),np.diff([y[peak],y[peaks[i-1]]]))
            if dist_prev < avg_peak_dist:
                near_peaks.append(peak)
        else:
            dist_next = norm(np.diff([x[peak],x[peaks[i+1]]]),np.diff([y[peak],y[peaks[i+1]]]))
            dist_prev = norm(np.diff([x[peak],x[peaks[i-1]]]),np.diff([y[peak],y[peaks[i-1]]]))
            if dist_next < avg_peak_dist or dist_prev < avg_peak_dist:
                near_peaks.append(peak)
    return near_peaks


def find_peaks_(x,y):
    """finds the peak with scipy function
    Saves the peaks of the original data that are an order higher than the noise.
    For the noise peaks, saves only the peaks of a convolution between a Gaussian distrib. with 10 std and the 
    original function.
    """
    peaks, _ = find_peaks(y)
    
    near_peaks = find_near_peaks(x,y,peaks)
    
    peaks_ = np.setdiff1d(peaks,near_peaks) # not near peaks

    gaussian = signal.windows.gaussian(len(y), std=10)
    
    convolution = signal.fftconvolve(y, gaussian, mode='same')

    peak_list, _ = find_peaks(convolution)

    peaks_ = np.append(peaks_,peak_list)
    
    peaks_ = [p for p in peaks_ if x[p] <= 20000] #deletes the high frequency peaks (higher than 20kHz)
    return peaks_
